## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_Database.csv")
city_data_df.head()

,City_ID,City,Country,Date,Lat,Lng,Current Description,Max Temp,Humidity %,Cloudiness %,Wind Speed
0,0,Ushuaia,AR,2022-02-07 23:22:02,-54.8000,-68.3000,scattered clouds,62.26,59,40,5.75
1,1,Hobart,AU,2022-02-07 23:26:37,-42.8794,147.3294,broken clouds,68.22,62,75,9.22
2,2,Port Alfred,ZA,2022-02-07 23:26:38,-33.5906,26.8910,few clouds,72.27,91,20,5.57
3,3,Ballina,AU,2022-02-07 23:26:38,-28.8667,153.5667,overcast clouds,73.58,90,100,12.66
4,4,Lebu,CL,2022-02-07 23:26:39,-37.6167,-73.6500,clear sky,62.11,82,3,14.63


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip?"))
max_temp = float(input("What is the maximum temperature you would like for your trip?"))

What is the minimum temperature you would like for your trip?75
What is the maximum temperature you would like for your trip?90


In [4]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] >= min_temp) & (city_data_df["Max Temp"] <= max_temp)]

In [5]:
# 4a. Determine if there are any empty rows - no missing empty rows
preferred_cities_df.count()

City_ID                184
City                   184
Country                184
Date                   184
Lat                    184
Lng                    184
Current Description    184
Max Temp               184
Humidity %             184
Cloudiness %           184
Wind Speed             184
dtype: int64

In [6]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
# no missing values

In [7]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = preferred_cities_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
5,Atuona,PF,77.79,light rain,-9.8000,-139.0333,
15,Kismayo,SO,77.32,overcast clouds,-0.3582,42.5454,
16,Phuket,TH,84.99,scattered clouds,8.0000,98.2500,
17,Sola,VU,81.88,moderate rain,-13.8833,167.5500,
18,Bandarbeyla,SO,75.06,few clouds,9.4942,50.8122,
21,Bambous Virieux,MU,77.25,broken clouds,-20.3428,57.7575,
23,Butaritari,KI,82.24,broken clouds,3.0707,172.7902,
31,Sulangan,PH,79.02,broken clouds,11.1373,123.7228,
33,Rikitea,PF,78.71,broken clouds,-23.1203,-134.9692,
43,Poya,NC,83.23,overcast clouds,-21.3500,165.1500,


In [9]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():

    # 6c. Get latitude and longitude from DataFrame, add location to params dict
    lat = row["Lat"]
    lng = row["Lng"]
    
    params['location'] = f"{lat},{lng}"

    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search.
    hotels = requests.get(base_url, params=params).json()
       
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except:
        print("Hotel not found... skipping.")
        

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [15]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df = hotel_df.dropna(how='any')
clean_hotel_df

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
5,Atuona,PF,77.79,light rain,-9.8000,-139.0333,Villa Enata
15,Kismayo,SO,77.32,overcast clouds,-0.3582,42.5454,Agan-Hotel
16,Phuket,TH,84.99,scattered clouds,8.0000,98.2500,The Surin
17,Sola,VU,81.88,moderate rain,-13.8833,167.5500,Leumerus Bungalows
18,Bandarbeyla,SO,75.06,few clouds,9.4942,50.8122,JABIR HOTEL
...,...,...,...,...,...,...,...
713,Oussouye,SN,75.38,scattered clouds,12.4850,-16.5469,Auberge du Routard
714,Caravelas,BR,77.70,scattered clouds,-17.7125,-39.2481,Pousada dos Navegantes
719,Almeirim,BR,75.49,broken clouds,-1.5233,-52.5817,Ricácio Hotel
723,Santarem,BR,77.13,few clouds,-2.4431,-54.7083,Ecotur Lodge


In [16]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [19]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Weather Description</dt><dd>{Current Description}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [22]:
# 11a. Add a marker layer for each city to the map. 
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)

# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))